In [ ]:
matplotlib notebook
import numpy as np
from scipy.stats import kde
from scipy.ndimage.filters import gaussian_filter
import pandas as pd
import os
import shutil
import subprocess
from subprocess import Popen
import datetime
import matplotlib.pyplot as plt  # plotting
import xlrd

from decimal import Decimal

from matplotlib.colors import LogNorm
from matplotlib.colors import SymLogNorm
from scipy.stats import binned_statistic_2d

# make prefix for figure filenames
now = datetime.datetime.now()
date = now.strftime('%Y%m%d')
pref = date

# Conversion: pixels to microns

In [17]:
pixels_to_um = 0.00006

# pixels_to_nm = 0.06

# list of all folders

In [1]:
all_paths = ['2018August_Tomo27/','2018June_Tomo14_Early_Invagination/','2018June_Tomo14_Late_Invagination/','2018June_Tomo26/','2018March/','2018November_32/']
all_paths

['2018August_Tomo27/',
 '2018June_Tomo14_Early_Invagination/',
 '2018June_Tomo14_Late_Invagination/',
 '2018June_Tomo26/',
 '2018March/',
 '2018November_32/']

In [4]:
# function: read a text file and get coordinate files
def read_text(file, label='fil'):
    
    import pandas as pd
    coordinatesfile = open(file, 'r')
    
    coords_df = pd.read_table(coordinatesfile, delim_whitespace=True)
#     check if there is an 'object' and 'contour' column (membrane)
    if len(coords_df.columns)==4:
        coords_df.columns=[label, 'X', 'Y', 'Z']
        coords_df=coords_df.set_index([label])
    
    elif len(coords_df.columns)==5:
        coords_df.columns=['object', label, 'X', 'Y', 'Z']
        coords_df=coords_df.set_index(['object', label])
        
    else:
        print("unexpected number of columns!")
#     make multiindex
    coordinatesfile.close()
    return coords_df


# Grab all unbranched filament files and put in a dataframe
## also convert to microns and add information about which dataset

In [25]:

all_unbranched_filaments = pd.DataFrame()
all_branched_filaments = pd.DataFrame()

for cur_path in all_paths:
    print(cur_path)
    files_in_dir = os.listdir(cur_path)
    for file in files_in_dir:
        if 'Branched' in file and 'txt' in file:
            branched_filaments_file = file
            branched_file_cur = cur_path+branched_filaments_file
            actin_br = read_text(branched_file_cur)
#             actin_br['X_um'] = actin_br['X']*pixels_to_um
            actin_br = actin_br.mul(pixels_to_um)
            actin_br['dataset'] = cur_path
            all_branched_filaments = pd.concat([all_branched_filaments, actin_br])


    for file in files_in_dir:
        if 'Unbranched' in file and 'txt' in file:
            unbranched_filaments_file = file
            unbranched_file_cur = cur_path + unbranched_filaments_file
            actin_ub = read_text(unbranched_file_cur)
            actin_ub = actin_ub.mul(pixels_to_um)
#             actin_ub['X_um'] = actin_ub['X']*pixels_to_um
#             actin_ub['Y_um'] = actin_ub['Y']*pixels_to_um
#             actin_ub['Z_um'] = actin_ub['Z']*pixels_to_um            
            actin_ub['dataset'] = cur_path
            all_unbranched_filaments = pd.concat([all_unbranched_filaments, actin_ub])


    # membrane = read_text(membranefile, 'contour')
    # membrane
all_unbranched_filaments

2018August_Tomo27/
2018June_Tomo14_Early_Invagination/
2018June_Tomo14_Late_Invagination/
2018June_Tomo26/
2018March/
2018November_32/


,X,Y,Z,dataset
fil,,,,
1,0.07956,0.07170,0.00510,2018August_Tomo27/
1,0.07926,0.06528,0.00600,2018August_Tomo27/
1,0.07908,0.06390,0.00606,2018August_Tomo27/
1,0.07920,0.06192,0.00612,2018August_Tomo27/
1,0.07890,0.05946,0.00564,2018August_Tomo27/
...,...,...,...,...
52,0.10554,0.03720,0.00264,2018November_32/
52,0.10458,0.04428,0.00294,2018November_32/
54,0.11322,0.03858,0.00522,2018November_32/


# Optionally convert to multi-index dataframe

In [35]:
all_unbranched_filaments_multi = all_unbranched_filaments.reset_index().set_index(['dataset','fil'])
all_unbranched_filaments_multi.sort_index(inplace=True)

all_branched_filaments_multi = all_branched_filaments.reset_index().set_index(['dataset','fil'])
all_branched_filaments_multi.sort_index(inplace=True)


all_unbranched_filaments_multi

X        Y        Z
dataset            fil                           
2018August_Tomo27/ 1    0.07956  0.07170  0.00510
                   1    0.07926  0.06528  0.00600
                   1    0.07908  0.06390  0.00606
                   1    0.07920  0.06192  0.00612
                   1    0.07890  0.05946  0.00564
...                         ...      ...      ...
2018November_32/   52   0.10554  0.03720  0.00264
                   52   0.10458  0.04428  0.00294
                   54   0.11322  0.03858  0.00522
                   54   0.11004  0.04050  0.00324
                   54   0.10698  0.04230  0.00138

[2002 rows x 3 columns]

In [36]:
# save
all_unbranched_filaments.to_csv('tomogram_cme_all_unbranched_coordinates.csv')
all_branched_filaments.to_csv('tomogram_cme_all_branched_coordinates.csv')

all_unbranched_filaments_multi.to_pickle('tomogram_cme_all_unbranched_coordinates.pkl')
all_branched_filaments_multi.to_pickle('tomogram_cme_all_branched_coordinates.pkl')